In [1]:
import math
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
from vocab import tokens

In [2]:
# gpt_moe_v2.py
import math
import torch
import torch.nn as nn
from vocab import tokens   # lista global de 209 tokens

class GPT(nn.Module):
    """
    GPT minimalista – 5 camadas, 4 cabeças, cada camada tem
    suas próprias projeções Q, K, V, Wo e seu próprio MLP.

    • forward(idx)         → logits (T, vocab_size)
    • predict_next_token() → string do próximo token
    • predict_all_sentence() → imprime passo-a-passo até max_tokens
    • init_tokens(prompt)  → guarda prompt inicial + vocabulário
    """

    def __init__(self, context_length, n_layers, n_heads, n_embd, max_tokens):
        super().__init__()

        # Hiper-parâmetros
        self.vocab_size     = 209
        self.context_length = context_length 
        self.n_layers       = n_layers
        self.n_heads        = n_heads
        self.n_embd         = n_embd
        self.max_tokens     = max_tokens
        self.head_dim       = self.n_embd // self.n_heads
        assert self.n_embd % self.n_heads == 0, "n_embd deve ser divisível por n_heads"

        # --- Embeddings -----------------------------------------------------
        self.wte = nn.Embedding(self.vocab_size, self.n_embd)
        self.wpe = nn.Embedding(self.context_length, self.n_embd)

        # --- Projeções Atenção & MLP independentes por camada ---------------
        self.qkv_proj = nn.ModuleList([
            nn.Linear(self.n_embd, 3 * self.n_embd) for _ in range(self.n_layers)
        ])
        self.out_proj = nn.ModuleList([
            nn.Linear(self.n_embd, self.n_embd) for _ in range(self.n_layers)
        ])
        self.mlp = nn.ModuleList([
            nn.Sequential(
                nn.Linear(self.n_embd, 2 * self.n_embd),
                nn.GELU(approximate='tanh'),
                nn.Linear(2 * self.n_embd, self.n_embd)
            ) for _ in range(self.n_layers)
        ])
        #self.ln1 = nn.ModuleList([nn.LayerNorm(self.n_embd) for _ in range(self.n_layers)])
        self.ln = nn.LayerNorm(self.n_embd)
        self.ln.weight.requires_grad = False   # desliga gradiente de γ
        self.ln.bias.requires_grad   = False   # desliga gradiente de β

        # --- Máscara causal (bool) ------------------------------------------
        self.register_buffer(
            "mask",
            torch.tril(torch.ones(self.context_length, self.context_length, dtype=torch.bool))
        )

        # --- Saída final -----------------------------------------------------
        #self.final_ln = nn.LayerNorm(self.n_embd)
        self.lm_head  = nn.Linear(self.n_embd, self.vocab_size)

        # --- Place-holders de prompt / vocabulário --------------------------

        self.token_to_idx = {tok: idx for idx, tok in enumerate(tokens)}
        self.idx_to_token = {idx: tok for tok, idx in self.token_to_idx.items()}

        # congelar parâmetros de todas as LayerNorm
        #for ln in list(self.ln1) + list(self.ln2) + [self.final_ln]:
        #    ln.weight.requires_grad = False   # desliga gradiente de γ
        #    ln.bias.requires_grad   = False   # desliga gradiente de β

    # --------------------------------------------------------------------- #
    #  Utilitários de vocabulário / prompt
    # --------------------------------------------------------------------- #
    def init_tokens(self, tokens_list):
        """Define prompt inicial e dicionários token↔id."""
        self.tokens_list  = list(tokens_list)


    def _update_indices_tensor(self, seq):
        """Converte `seq` (lista de tokens) para tensor de índices,
           limitando ao último `context_length`."""
        idxs = [self.token_to_idx[tok] for tok in seq][-self.context_length:]
        return torch.tensor(idxs, dtype=torch.long, device=self.wte.weight.device)

    # --------------------------------------------------------------------- #
    #  Bloco de atenção (usa pesos da camada `l`)
    # --------------------------------------------------------------------- #
    def _self_attention(self, x, l):
        # x: (T, C)
        T, C = x.size()
        qkv = self.qkv_proj[l](x)                 # (T, 3C)
        q, k, v = qkv.chunk(3, dim=1)

        q = q.view(T, self.n_heads, self.head_dim).transpose(0, 1)  # (nh,T,hd)
        k = k.view(T, self.n_heads, self.head_dim).transpose(0, 1)
        v = v.view(T, self.n_heads, self.head_dim).transpose(0, 1)

        att = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        att = att.masked_fill(~self.mask[:T, :T], float('-inf'))
        att = torch.softmax(att, dim=-1)
        y   = att @ v                                           # (nh,T,hd)
        y   = y.transpose(0, 1).contiguous().view(T, C)         # (T,C)
        return self.out_proj[l](y)                              # (T,C)

    # --------------------------------------------------------------------- #
    #  Forward: devolve logits para todos os tokens
    # --------------------------------------------------------------------- #
    def forward(self, idx):
        """
        idx : LongTensor (T,) – sequência de índices de tokens.
        Retorna logits (T, vocab_size).
        """
        T = idx.size(0)
        pos = torch.arange(T, device=idx.device)
        #print(idx)
        #print(pos)
        x   = self.wte(idx) + self.wpe(pos)

        for l in range(self.n_layers):
            x = x + self._self_attention(self.ln(x), l)
            x = x + self.mlp[l](self.ln(x))

        x = self.ln(x)                      # (T, C)
        return self.lm_head(x)                    # (T, vocab_size)

    # --------------------------------------------------------------------- #
    #  Geração
    # --------------------------------------------------------------------- #
    def predict_next_token(self, seq):
        """
        seq : lista de tokens (prompt + já gerados)
        Retorna o próximo token (string).
        """
        idx = self._update_indices_tensor(seq)
        logits = self.forward(idx)
        probs  = torch.softmax(logits[-1], dim=-1)       # último passo
        next_id = torch.multinomial(probs, 1).item()
        return self.idx_to_token[next_id]

    def predict_all_sentence(self):
        """
        Gera até `max_tokens` novos tokens SEM alterar o prompt original.
        Imprime a frase a cada passo e devolve a lista completa.
        """
        prompt = list(self.tokens_list)          # cópia imutável
        generated = []

        for step in range(self.max_tokens):
            next_tok = self.predict_next_token(prompt + generated)
            generated.append(next_tok)
            print(f"Passo {step+1}: {' '.join(prompt + generated)}")

        return prompt + generated



In [3]:
list(enumerate(tokens))

[(0, 'eu'),
 (1, 'você'),
 (2, 'ele'),
 (3, 'ela'),
 (4, 'nós'),
 (5, 'vocês'),
 (6, 'eles'),
 (7, 'elas'),
 (8, 'me'),
 (9, 'te'),
 (10, 'se'),
 (11, 'nos'),
 (12, 'vos'),
 (13, 'lhe'),
 (14, 'lhes'),
 (15, 'mim'),
 (16, 'ti'),
 (17, 'si'),
 (18, 'ser'),
 (19, 'estar'),
 (20, 'ter'),
 (21, 'haver'),
 (22, 'fazer'),
 (23, 'poder'),
 (24, 'dizer'),
 (25, 'ver'),
 (26, 'dar'),
 (27, 'saber'),
 (28, 'querer'),
 (29, 'chegar'),
 (30, 'passar'),
 (31, 'dever'),
 (32, 'ficar'),
 (33, 'deixar'),
 (34, 'pensar'),
 (35, 'vir'),
 (36, 'conhecer'),
 (37, 'casa'),
 (38, 'tempo'),
 (39, 'dia'),
 (40, 'mundo'),
 (41, 'homem'),
 (42, 'mulher'),
 (43, 'vida'),
 (44, 'mão'),
 (45, 'olho'),
 (46, 'palavra'),
 (47, 'caminho'),
 (48, 'gato'),
 (49, 'cachorro'),
 (50, 'carro'),
 (51, 'livro'),
 (52, 'porta'),
 (53, 'rua'),
 (54, 'trabalho'),
 (55, 'dinheiro'),
 (56, 'noite'),
 (57, 'bom'),
 (58, 'mau'),
 (59, 'feliz'),
 (60, 'triste'),
 (61, 'grande'),
 (62, 'pequeno'),
 (63, 'novo'),
 (64, 'velho'),
 (65,

In [4]:
context_length = 20
n_layers       = 3
n_heads        = 2
n_embd         = 64
max_tokens     = 12
tokens_list = ["o", "gato", "pequeno"]
model = GPT(context_length, n_layers, n_heads, n_embd, max_tokens)
model.init_tokens(tokens_list)

In [5]:
model.predict_all_sentence()

Passo 1: o gato pequeno perto
Passo 2: o gato pequeno perto história
Passo 3: o gato pequeno perto história livro
Passo 4: o gato pequeno perto história livro quatro
Passo 5: o gato pequeno perto história livro quatro este
Passo 6: o gato pequeno perto história livro quatro este com
Passo 7: o gato pequeno perto história livro quatro este com conhecer
Passo 8: o gato pequeno perto história livro quatro este com conhecer país
Passo 9: o gato pequeno perto história livro quatro este com conhecer país ontem
Passo 10: o gato pequeno perto história livro quatro este com conhecer país ontem vida
Passo 11: o gato pequeno perto história livro quatro este com conhecer país ontem vida está
Passo 12: o gato pequeno perto história livro quatro este com conhecer país ontem vida está foi


['o',
 'gato',
 'pequeno',
 'perto',
 'história',
 'livro',
 'quatro',
 'este',
 'com',
 'conhecer',
 'país',
 'ontem',
 'vida',
 'está',
 'foi']

In [ ]:
model.idx_to_token[93]

In [ ]:
for name, param in model.named_parameters():
    print(f"Nome: {name}")
    print(f"Shape: {param.shape}")
    print(f"Valores: {param}\n")


In [ ]:
print(model)

In [ ]:
total = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total treinável: {total:,d}")


In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name:<40} {param.numel():>10,}")


In [ ]:
model.predict_next_token()

In [ ]:
model.predict_all_sentence()